In [87]:
import cv2
import numpy as np

net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)

distance_thres = 50
image = cv2.imread("test_image3.jpg")

In [88]:
height, width = image.shape[:2]

blob = cv2.dnn.blobFromImage(image, 0.00392, (416, 416), (0, 0, 0), True, crop=False)

net.setInput(blob)
layer_names = net.getUnconnectedOutLayersNames()
outs = net.forward(layer_names)

In [89]:
classes = [] # classes list
with open("coco.names", "r") as f:
  classes = [line.strip() for line in f.readlines()] 
print(classes)

['person', 'bicycle', 'car', 'motorbike', 'aeroplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'sofa', 'pottedplant', 'bed', 'diningtable', 'toilet', 'tvmonitor', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']


In [90]:
confidences = []
boxes = []
# class_ids = []
# print(outs[0])
for out in outs:
    for detection in out:
        scores = detection[5:]
#         print(scores.shape)
        class_id = np.argmax(scores)
        if class_id != 0:
            continue
        confidence = scores[class_id]
        if confidence > 0.3:
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)
            x = int(center_x - w / 2)
            y = int(center_y - h / 2)
            boxes.append([x, y, w, h])
            confidences.append(float(confidence))
#             class_ids.append(class_id)

indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.3, 0.45)

In [91]:
persons = []
person_centres = []
violate = set()

for i in range(len(boxes)):
    if i in indexes:
        x, y, w, h = boxes[i]
        persons.append(boxes[i])
        person_centres.append([x + w // 2, y + h // 2])



In [92]:
for i in range(len(persons)):
    for j in range(i + 1, len(persons)):
        if np.linalg.norm(np.array(person_centres[i]) - np.array(person_centres[j])) <= distance_thres:
            violate.add(tuple(persons[i]))
            violate.add(tuple(persons[j]))

In [93]:
for (x, y, w, h) in persons:
    if (x, y, w, h) in violate:
        color = (0, 0, 255)
    else:
        color = (0, 255, 0)
    cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)
    cv2.circle(image, (x + w // 2, y + h // 2), 5, (0, 0, 255), 2)

In [94]:
cv2.putText(image, f'No of Violations: {len(violate)}', (15, height - 10),
            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 126, 255), 2)

cv2.imshow("Social Distance Detection", image)
cv2.waitKey(0)
cv2.destroyAllWindows()